## import libraries and clean data

In [1]:
import pandas as pd
import re
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.concat(map(pd.read_csv,['aqi_quality_0.csv', 'aqi_quality_1.csv', 'aqi_quality_2.csv', 'aqi_quality_3.csv', 'aqi_quality_4.csv', 'aqi_quality_5.csv', 'aqi_quality_6.csv', 'aqi_quality_7.csv', 'aqi_quality_8.csv', 'aqi_quality_9.csv', 'aqi_quality_10.csv', 'aqi_quality_11.csv','aqi_quality_12.csv', 'aqi_quality_13.csv', 'aqi_quality_14.csv', 'aqi_quality_15.csv', 'aqi_quality_16.csv']))
df.head(5)

,Province,City,County,Date,AQI
0,安徽,合肥,合肥,2019-12-31,56.0
1,安徽,合肥,包河,2019-12-31,67.0
2,安徽,合肥,长丰,2019-12-31,47.0
3,安徽,合肥,巢湖,2019-12-31,44.0
4,安徽,合肥,肥东,2019-12-31,45.0


In [3]:
df.shape

(4539065, 5)

In [4]:
df.describe()

,AQI
count,4.250666e+06
mean,5.342410e+01
std,4.195701e+01
min,-9.900000e+01
25%,2.700000e+01
50%,4.400000e+01
75%,6.800000e+01
max,5.000000e+02


Then to check whether there is row which does not have an aqi. if there is, delete it.

In [3]:
df.isnull().any()

Province    False
City        False
County      False
Date        False
AQI          True
dtype: bool

Yes, there is.

In [4]:
df = df[df.AQI >= 0]

In [5]:
df.isnull().any()

Province    False
City        False
County      False
Date        False
AQI         False
dtype: bool

Now there is no row without AQI :)

## at province level, daily average, median, 25% percentile, 75 percentile

In [ ]:
for name, group in df.groupby('Province'):
    group.to_csv('{}.csv'.format(name), index=False)

group by province and generate separate csvs for referring

In [ ]:
for name, group in df.groupby('Province'):
    grouped = group.groupby('Date')['AQI'].mean()
    mean_name = name + "mean_daily_aqi"
    grouped.plot(kind='bar', title=name + 'daily average', figsize=(400,225))
    plt.title(name+"Daily AQI Mean",fontsize=400,color='r')
    plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.savefig(mean_name + '.pdf')

In [ ]:
for name, group in df.groupby('Province'):
    grouped = group.groupby('Date')['AQI'].median()
    median_name = name + "median_daily_aqi"
    grouped.plot(kind='bar', title=name + 'daily median', figsize=(400,225))
    plt.title(name+"Daily AQI Median",fontsize=400,color='r')
    plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.savefig(median_name + '.pdf')

In [ ]:
for name, group in df.groupby('Province'):
    grouped = group.groupby('Date')['AQI'].quantile(.25)
    placeholder = name + "25%_daily_aqi"
    grouped.plot(kind='bar', title=name + 'daily 25% aqi', figsize=(400,225))
    plt.title(name+"Daily AQI 25%",fontsize=400,color='r')
    plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.savefig(placeholder + '.pdf')

In [ ]:
for name, group in df.groupby('Province'):
    grouped = group.groupby('Date')['AQI'].quantile(.75)
    placeholder = name + "75%_daily_aqi"
    grouped.plot(kind='bar', title=name + 'daily 75% aqi', figsize=(400,225))
    plt.title(name+"Daily AQI 75%",fontsize=400,color='r')
    plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
    plt.savefig(placeholder + '.pdf')

## plot annual aqi map on county level


In [14]:
import numpy as np
from pyecharts import Geo
from pyecharts import Map
from pyecharts.datasets.coordinates import get_coordinate,search_coordinates_by_keyword

firstly, add a year column for groupby application

In [15]:
df['year'] = pd.DatetimeIndex(df['Date']).year

In [16]:
df.head(20)

,Province,City,County,Date,AQI,year
0,安徽,合肥,合肥,2019-12-31,56.0,2019
1,安徽,合肥,包河,2019-12-31,67.0,2019
2,安徽,合肥,长丰,2019-12-31,47.0,2019
3,安徽,合肥,巢湖,2019-12-31,44.0,2019
4,安徽,合肥,肥东,2019-12-31,45.0,2019
5,安徽,合肥,肥西,2019-12-31,61.0,2019
6,安徽,安庆,安庆,2019-12-31,74.0,2019
7,安徽,亳州,涡阳,2019-12-31,40.0,2019
8,安徽,亳州,亳州,2019-12-31,52.0,2019
9,安徽,亳州,利辛,2019-12-31,43.0,2019


second, collect county name and AQI mean by year

In [17]:
indexs = []
values = []
for name, group in df[df.year==2019].groupby('County'):
    indexs.append(format(name))
    values.append(group['AQI'].mean())

then, write a remove_none function to clean unfound cities

In [18]:
def remove_None(areas,values): 
    None_areas = []
    i = 0
    while(i != len(areas)):
        if search_coordinates_by_keyword(areas[i][:2]) == {}:
            None_areas.append(areas[i])
            areas.remove(areas[i])
            values.remove(values[i])
        else:
            areas[i] = search_coordinates_by_keyword(areas[i][:2]).popitem()[0]
            i = i + 1
    print('无效区域：',None_areas)
    return areas,values

In [19]:
remove_None(indexs, values)

无效区域： ['七星', '七星关', '万秀', '三峡', '三沙', '上党区', '东岗', '东沟', '中沙', '中甸', '义安', '九华山', '云冈区', '云州区', '伊州', '保德', '元江', '克州', '八步', '八里罕', '六库', '兰陵', '兴宾', '冷湖', '利州', '加格达奇', '北屯', '北镇', '华州', '南沙', '南沙岛', '南湖', '博克图', '博州', '博望', '卡若', '双江', '双河', '双湖', '叠彩', '可克达拉', '右江', '叶集', '吉兰太', '吉阳', '吴旗', '吴起', '呼中', '图里河', '大佘太', '大柴旦', '天池', '天涯', '太华山', '姑苏', '威宁', '嫩江', '孟连', '孪井滩', '宁冈', '宁洱', '安州', '宝国吐', '小二沟', '岗子', '崇武', '崖州', '川汇', '巴仑台', '巴宜', '巴里坤', '巴雅尔吐胡硕', '帕里', '平城区', '平桂', '平鲁', '开州', '成山头', '拐子湖', '播州', '新吴', '新界', '昭化', '普洱', '曹妃甸', '朱日和', '杜蒙', '柯桥', '柳北', '桑珠孜', '桥东', '桥西', '梁溪', '梅里斯', '榕城', '氹仔岛', '江州', '江浦', '江海', '江源', '沈北新区', '泽当', '洪家', '浑南', '浑江', '浩尔吐', '海力素', '海棠', '海陵', '涠洲岛', '淄川', '港北', '港南', '湘江新区', '潞州区', '濠江', '瀍河', '灵邱', '炮台', '狮泉河', '玉州', '瓜州', '瓯海', '白杨沟', '盘州', '石岛', '碧江', '碾子山', '祥符', '禅城', '福绵', '禹王台', '秀峰', '秦州', '积石山', '米东', '索伦', '红塔', '红寺堡', '耀县', '胡尔勒', '舍伯吐', '芒市', '花溪', '茫崖', '荔浦', '莫力达瓦旗', '莫索湾', '蓟州', '蓬江', '蔡家湖', '襄州', '西沙', '观山湖', '路环岛', '那仁宝力格

(['丁青县',
  '七台河市',
  '万宁市',
  '万安县',
  '万山特区',
  '万州区',
  '万年县',
  '万柏林区',
  '万盛区',
  '万荣县',
  '万载县',
  '三亚市',
  '三元区',
  '三台县',
  '三明市',
  '三水区',
  '三江侗族自治县',
  '三穗县',
  '三都水族自治县',
  '三门峡市',
  '上城区',
  '上思县',
  '上杭县',
  '上林县',
  '上栗县',
  '上海',
  '上犹县',
  '上甘岭区',
  '上虞市',
  '上街区',
  '上饶县',
  '上饶县',
  '上高县',
  '下城区',
  '下陆区',
  '丘北县',
  '丛台区',
  '东丰县',
  '东丽区',
  '东乌珠穆沁旗',
  '东乡族自治县',
  '东兰县',
  '东兴区',
  '城东区',
  '东台市',
  '东坡区',
  '东城区',
  '东宁县',
  '东安县',
  '东宝区',
  '东山县',
  '东川区',
  '东平县',
  '东方市',
  '东昌府区',
  '东明县',
  '东洲区',
  '东海县',
  '东港区',
  '东湖区',
  '东源县',
  '东胜区',
  '东至县',
  '东莞市',
  '东营区',
  '东西湖区',
  '东辽县',
  '东阳市',
  '东阿县',
  '东陵区',
  '东风区',
  '两当县',
  '个旧市',
  '中卫市',
  '中原区',
  '中宁县',
  '中山市',
  '中方县',
  '中江县',
  '中牟县',
  '中阳县',
  '丰南区',
  '民丰县',
  '丰台区',
  '丰城市',
  '丰泽区',
  '丰润区',
  '丰满区',
  '丰都县',
  '丰镇市',
  '丰顺县',
  '临县',
  '临夏县',
  '临安市',
  '临川区',
  '临朐县',
  '临桂县',
  '临武县',
  '临江市',
  '临汾市',
  '临沂市',
  '临沧市',
  '临沭县',
  '临河区',
  '临泽县',
  '临洮县',
  '临海市',
  '临淄区',
  '临清市',


In [25]:
geo = Geo(
    "2019中国空气质量指数",
    "first two coordinates are latitude and longitude, last coordinate is AQI",
    title_color="#ffffff",
    title_pos="center",
    width=1200,
    height=600,
    background_color="#404a59",
)
geo.add(
    "",
    indexs,
    values,
visual_range=[0, 250], visual_text_color="#fff",
        symbol_size=5, is_visualmap=True)
geo.show_config()
geo.render(path="2019中国空气质量指数.html")


{
    "title": [
        {
            "text": "2019\u4e2d\u56fd\u7a7a\u6c14\u8d28\u91cf\u6307\u6570",
            "subtext": "first two coordinates are latitude and longitude, last coordinate is AQI",
            "left": "center",
            "top": "auto",
            "textStyle": {
                "color": "#ffffff",
                "fontSize": 18
            },
            "subtextStyle": {
                "fontSize": 12
            }
        }
    ],
    "toolbox": {
        "show": true,
        "orient": "vertical",
        "left": "95%",
        "top": "center",
        "feature": {
            "saveAsImage": {
                "show": true,
                "title": "save as image"
            },
            "restore": {
                "show": true,
                "title": "restore"
            },
            "dataView": {
                "show": true,
                "title": "data view"
            }
        }
    },
    "series_id": 5989107,
    "tooltip": {
        "trigg

In [26]:
indexs = []
values = []
for name, group in df[df.year==2018].groupby('County'):
    indexs.append(format(name))
    values.append(group['AQI'].mean())

In [27]:
remove_None(indexs, values)

无效区域： ['七星', '七星关', '万秀', '三峡', '三沙', '上党区', '东岗', '东沟', '中沙', '中甸', '义安', '云冈区', '云州区', '伊州', '保德', '元江', '克州', '八步', '八里罕', '六库', '兰陵', '兴宾', '冷湖', '利州', '加格达奇', '北屯', '北镇', '华州', '南沙', '南沙岛', '南湖', '博克图', '博州', '博望', '卡若', '双江', '双河', '双湖', '叠彩', '可克达拉', '右江', '叶集', '吉兰太', '吉阳', '吴旗', '吴起', '呼中', '图里河', '大佘太', '大柴旦', '天涯', '太华山', '姑苏', '威宁', '嫩江', '孟连', '孪井滩', '宁冈', '宁洱', '安州', '宝国吐', '小二沟', '岗子', '崇武', '崖州', '川汇', '巴仑台', '巴宜', '巴里坤', '巴雅尔吐胡硕', '帕里', '平城区', '平桂', '平鲁', '开州', '成山头', '拐子湖', '播州', '新吴', '新界', '昭化', '普洱', '曹妃甸', '朱日和', '杜蒙', '柯桥', '柳北', '桑珠孜', '桥东', '桥西', '梁溪', '梅里斯', '榕城', '氹仔岛', '江州', '江浦', '江海', '江源', '沈北新区', '泽当', '洪家', '浑南', '浑江', '浩尔吐', '海力素', '海棠', '海陵', '涠洲岛', '淄川', '港北', '港南', '湘江新区', '潞州区', '濠江', '瀍河', '灵邱', '炮台', '狮泉河', '玉州', '瓜州', '瓯海', '白杨沟', '盘州', '石岛', '碧江', '碾子山', '祥符', '禅城', '福绵', '禹王台', '秀峰', '秦州', '积石山', '米东', '索伦', '红塔', '红寺堡', '耀县', '胡尔勒', '舍伯吐', '芒市', '花溪', '茫崖', '荔浦', '莫力达瓦旗', '莫索湾', '蓟州', '蓬江', '蔡家湖', '襄州', '西沙', '观山湖', '路环岛', '那仁宝力格', '邱北', '郧阳'

(['丁青县',
  '七台河市',
  '万宁市',
  '万安县',
  '万山特区',
  '万州区',
  '万年县',
  '万柏林区',
  '万盛区',
  '万荣县',
  '万载县',
  '三亚市',
  '三元区',
  '三台县',
  '三明市',
  '三水区',
  '三江侗族自治县',
  '三穗县',
  '三都水族自治县',
  '三门峡市',
  '上城区',
  '上思县',
  '上杭县',
  '上林县',
  '上栗县',
  '上海',
  '上犹县',
  '上甘岭区',
  '上虞市',
  '上饶县',
  '上饶县',
  '上高县',
  '下城区',
  '下陆区',
  '丘北县',
  '丛台区',
  '东丰县',
  '东丽区',
  '东乌珠穆沁旗',
  '东乡族自治县',
  '东兰县',
  '东兴区',
  '城东区',
  '东台市',
  '东坡区',
  '东城区',
  '东宁县',
  '东安县',
  '东宝区',
  '东山县',
  '东川区',
  '东平县',
  '东方市',
  '东昌府区',
  '东明县',
  '东洲区',
  '东港区',
  '东湖区',
  '东源县',
  '东胜区',
  '东至县',
  '东莞市',
  '东营区',
  '东西湖区',
  '东辽县',
  '东阳市',
  '东阿县',
  '东陵区',
  '东风区',
  '两当县',
  '个旧市',
  '中卫市',
  '中原区',
  '中宁县',
  '中山市',
  '中方县',
  '中江县',
  '中牟县',
  '中阳县',
  '丰南区',
  '民丰县',
  '丰台区',
  '丰城市',
  '丰泽区',
  '丰润区',
  '丰满区',
  '丰都县',
  '丰镇市',
  '丰顺县',
  '临县',
  '临夏县',
  '临安市',
  '临川区',
  '临朐县',
  '临桂县',
  '临武县',
  '临江市',
  '临汾市',
  '临沂市',
  '临沧市',
  '临沭县',
  '临河区',
  '临泽县',
  '临洮县',
  '临海市',
  '临淄区',
  '临清市',
  '临渭区',
  '临湘市',


In [28]:
geo = Geo(
    "2018中国空气质量指数",
    "first two coordinates are latitude and longitude, last coordinate is AQI",
    title_color="#ffffff",
    title_pos="center",
    width=1200,
    height=600,
    background_color="#404a59",
)
geo.add(
    "",
    indexs,
    values,
visual_range=[0, 250], visual_text_color="#fff",
        symbol_size=5, is_visualmap=True)
geo.show_config()
geo.render(path="2018中国空气质量指数.html")


{
    "title": [
        {
            "text": "2018\u4e2d\u56fd\u7a7a\u6c14\u8d28\u91cf\u6307\u6570",
            "subtext": "first two coordinates are latitude and longitude, last coordinate is AQI",
            "left": "center",
            "top": "auto",
            "textStyle": {
                "color": "#ffffff",
                "fontSize": 18
            },
            "subtextStyle": {
                "fontSize": 12
            }
        }
    ],
    "toolbox": {
        "show": true,
        "orient": "vertical",
        "left": "95%",
        "top": "center",
        "feature": {
            "saveAsImage": {
                "show": true,
                "title": "save as image"
            },
            "restore": {
                "show": true,
                "title": "restore"
            },
            "dataView": {
                "show": true,
                "title": "data view"
            }
        }
    },
    "series_id": 7406821,
    "tooltip": {
        "trigg

In [31]:
indexs = []
values = []
for name, group in df[df.year==2017].groupby('County'):
    indexs.append(format(name))
    values.append(group['AQI'].mean())

In [32]:
remove_None(indexs, values)

无效区域： ['七星', '七星关', '万秀', '三峡', '三沙', '上党区', '东岗', '东沟', '中沙', '中甸', '义安', '云冈区', '云州区', '伊州', '保德', '元江', '克州', '八步', '八里罕', '六库', '兰陵', '兴宾', '冷湖', '利州', '加格达奇', '北屯', '北镇', '华州', '南沙', '南沙岛', '南湖', '博克图', '博州', '博望', '卡若', '双江', '双河', '双湖', '叠彩', '可克达拉', '右江', '叶集', '吉兰太', '吉阳', '吴旗', '吴起', '呼中', '图里河', '大佘太', '大柴旦', '天涯', '太华山', '姑苏', '威宁', '嫩江', '孟连', '孪井滩', '宁冈', '宁洱', '安州', '宝国吐', '小二沟', '岗子', '崇武', '崖州', '川汇', '巴仑台', '巴宜', '巴里坤', '巴雅尔吐胡硕', '帕里', '平城区', '平桂', '平鲁', '开州', '成山头', '拐子湖', '播州', '新吴', '新界', '昭化', '普洱', '曹妃甸', '朱日和', '杜蒙', '柯桥', '柳北', '桑珠孜', '桥东', '桥西', '梁溪', '梅里斯', '榕城', '氹仔岛', '江州', '江浦', '江海', '江源', '沈北新区', '泽当', '洪家', '浑南', '浑江', '浩尔吐', '海力素', '海棠', '海陵', '涠洲岛', '淄川', '港北', '港南', '湘江新区', '潞州区', '濠江', '瀍河', '灵邱', '炮台', '狮泉河', '玉州', '瓜州', '瓯海', '白杨沟', '盘州', '石岛', '碧江', '碾子山', '祥符', '禅城', '福绵', '禹王台', '秀峰', '秦州', '积石山', '米东', '索伦', '红塔', '红寺堡', '耀县', '胡尔勒', '舍伯吐', '芒市', '花溪', '茫崖', '荔浦', '莫力达瓦旗', '莫索湾', '蓟州', '蓬江', '蔡家湖', '襄州', '西沙', '观山湖', '路环岛', '那仁宝力格', '邱北', '郧阳'

(['丁青县',
  '七台河市',
  '万宁市',
  '万安县',
  '万山特区',
  '万州区',
  '万年县',
  '万柏林区',
  '万盛区',
  '万荣县',
  '万载县',
  '三亚市',
  '三元区',
  '三台县',
  '三明市',
  '三水区',
  '三江侗族自治县',
  '三穗县',
  '三都水族自治县',
  '三门峡市',
  '上城区',
  '上思县',
  '上杭县',
  '上林县',
  '上栗县',
  '上海',
  '上犹县',
  '上甘岭区',
  '上虞市',
  '上饶县',
  '上饶县',
  '上高县',
  '下城区',
  '下陆区',
  '丘北县',
  '丛台区',
  '东丰县',
  '东丽区',
  '东乌珠穆沁旗',
  '东乡族自治县',
  '东兰县',
  '东兴区',
  '城东区',
  '东台市',
  '东坡区',
  '东城区',
  '东宁县',
  '东安县',
  '东宝区',
  '东山县',
  '东川区',
  '东平县',
  '东方市',
  '东昌府区',
  '东明县',
  '东洲区',
  '东港区',
  '东湖区',
  '东源县',
  '东胜区',
  '东至县',
  '东莞市',
  '东营区',
  '东西湖区',
  '东辽县',
  '东阳市',
  '东阿县',
  '东陵区',
  '东风区',
  '两当县',
  '个旧市',
  '中卫市',
  '中原区',
  '中宁县',
  '中山市',
  '中方县',
  '中江县',
  '中牟县',
  '中阳县',
  '丰南区',
  '民丰县',
  '丰台区',
  '丰城市',
  '丰泽区',
  '丰润区',
  '丰满区',
  '丰都县',
  '丰镇市',
  '丰顺县',
  '临县',
  '临夏县',
  '临安市',
  '临川区',
  '临朐县',
  '临桂县',
  '临武县',
  '临江市',
  '临汾市',
  '临沂市',
  '临沧市',
  '临沭县',
  '临河区',
  '临泽县',
  '临洮县',
  '临海市',
  '临淄区',
  '临渭区',
  '临湘市',
  '临潭县',


In [33]:
geo = Geo(
    "2017中国空气质量指数",
    "first two coordinates are latitude and longitude, last coordinate is AQI",
    title_color="#ffffff",
    title_pos="center",
    width=1200,
    height=600,
    background_color="#404a59",
)
geo.add(
    "",
    indexs,
    values,
visual_range=[0, 250], visual_text_color="#fff",
        symbol_size=5, is_visualmap=True)
geo.show_config()
geo.render(path="2017中国空气质量指数.html")


{
    "title": [
        {
            "text": "2017\u4e2d\u56fd\u7a7a\u6c14\u8d28\u91cf\u6307\u6570",
            "subtext": "first two coordinates are latitude and longitude, last coordinate is AQI",
            "left": "center",
            "top": "auto",
            "textStyle": {
                "color": "#ffffff",
                "fontSize": 18
            },
            "subtextStyle": {
                "fontSize": 12
            }
        }
    ],
    "toolbox": {
        "show": true,
        "orient": "vertical",
        "left": "95%",
        "top": "center",
        "feature": {
            "saveAsImage": {
                "show": true,
                "title": "save as image"
            },
            "restore": {
                "show": true,
                "title": "restore"
            },
            "dataView": {
                "show": true,
                "title": "data view"
            }
        }
    },
    "series_id": 5175379,
    "tooltip": {
        "trigg

In [34]:
indexs = []
values = []
for name, group in df[df.year==2016].groupby('County'):
    indexs.append(format(name))
    values.append(group['AQI'].mean())

In [35]:
remove_None(indexs, values)

无效区域： ['三峡', '上党区', '东岗', '东沟', '中沙', '中甸', '云冈区', '云州区', '保德', '元江', '克州', '八里罕', '六库', '兰陵', '冷湖', '加格达奇', '北镇', '南沙岛', '博克图', '博州', '双江', '双湖', '吉兰太', '吴旗', '吴起', '呼中', '图里河', '大佘太', '大柴旦', '天池', '太华山', '威宁', '嫩江', '孟连', '孪井滩', '宁冈', '宁洱', '宝国吐', '小二沟', '岗子', '崇武', '巴仑台', '巴里坤', '巴雅尔吐胡硕', '帕里', '平城区', '平鲁', '成山头', '拐子湖', '新界', '普洱', '曹妃甸', '朱日和', '杜蒙', '柯桥', '氹仔岛', '江浦', '江海', '江源', '泽当', '洪家', '浩尔吐', '海力素', '涠洲岛', '淄川', '湘江新区', '潞州区', '灵邱', '炮台', '狮泉河', '瓜州', '白杨沟', '盘州', '石岛', '积石山', '索伦', '耀县', '胡尔勒', '舍伯吐', '芒市', '花溪', '茫崖', '荔浦', '莫力达瓦旗', '莫索湾', '蓟州', '蓬江', '蔡家湖', '襄州', '西沙', '路环岛', '那仁宝力格', '邱北', '郧阳', '郫都', '钓鱼岛', '镇沅', '陈旗', '陶乐', '隆昌', '随县', '霍尔果斯', '高力板', '麦积', '黔阳', '龙感湖管理区']


(['丁青县',
  '七台河市',
  '万宁市',
  '万安县',
  '万山特区',
  '万州区',
  '万年县',
  '万柏林区',
  '万盛区',
  '万荣县',
  '万载县',
  '三亚市',
  '三台县',
  '三明市',
  '三水区',
  '三江侗族自治县',
  '三穗县',
  '三都水族自治县',
  '三门峡市',
  '上思县',
  '上杭县',
  '上林县',
  '上栗县',
  '上海',
  '上犹县',
  '上虞市',
  '上街区',
  '上饶县',
  '上饶县',
  '上高县',
  '下陆区',
  '丘北县',
  '东丰县',
  '东丽区',
  '东乌珠穆沁旗',
  '东乡族自治县',
  '东兰县',
  '东兴区',
  '东台市',
  '东宁县',
  '东宝区',
  '东山县',
  '东川区',
  '东平县',
  '东方市',
  '东明县',
  '东海县',
  '东港区',
  '东源县',
  '东胜区',
  '东至县',
  '东莞市',
  '东营区',
  '东西湖区',
  '东辽县',
  '东阳市',
  '东阿县',
  '两当县',
  '个旧市',
  '中卫市',
  '中宁县',
  '中山市',
  '中方县',
  '中江县',
  '中牟县',
  '中阳县',
  '丰南区',
  '民丰县',
  '丰台区',
  '丰城市',
  '丰润区',
  '丰都县',
  '丰镇市',
  '丰顺县',
  '临县',
  '临夏县',
  '临安市',
  '临朐县',
  '临桂县',
  '临武县',
  '临江市',
  '临汾市',
  '临沂市',
  '临沧市',
  '临沭县',
  '临河区',
  '临泽县',
  '临洮县',
  '临海市',
  '临淄区',
  '临湘市',
  '临潭县',
  '临猗县',
  '临高县',
  '丹东市',
  '丹凤县',
  '丹寨县',
  '丹巴县',
  '丹徒区',
  '丹棱县',
  '丹江口市',
  '丹阳市',
  '丽水市',
  '丽江市',
  '乃东县',
  '久治县',
  '义乌市',
  '遵义县',
  '义马市',
 

In [36]:
geo = Geo(
    "2016中国空气质量指数",
    "first two coordinates are latitude and longitude, last coordinate is AQI",
    title_color="#ffffff",
    title_pos="center",
    width=1200,
    height=600,
    background_color="#404a59",
)
geo.add(
    "",
    indexs,
    values,
visual_range=[0, 250], visual_text_color="#fff",
        symbol_size=5, is_visualmap=True)
geo.show_config()
geo.render(path="2016中国空气质量指数.html")


{
    "title": [
        {
            "text": "2016\u4e2d\u56fd\u7a7a\u6c14\u8d28\u91cf\u6307\u6570",
            "subtext": "first two coordinates are latitude and longitude, last coordinate is AQI",
            "left": "center",
            "top": "auto",
            "textStyle": {
                "color": "#ffffff",
                "fontSize": 18
            },
            "subtextStyle": {
                "fontSize": 12
            }
        }
    ],
    "toolbox": {
        "show": true,
        "orient": "vertical",
        "left": "95%",
        "top": "center",
        "feature": {
            "saveAsImage": {
                "show": true,
                "title": "save as image"
            },
            "restore": {
                "show": true,
                "title": "restore"
            },
            "dataView": {
                "show": true,
                "title": "data view"
            }
        }
    },
    "series_id": 5923856,
    "tooltip": {
        "trigg

## calculate national daily average, median, 25 percentile, 75 percentile and plot

In [ ]:
grouped = df.groupby('Date')['AQI'].mean()

In [ ]:
grouped.describe()

In [ ]:
grouped.plot(kind='bar', title='national daily average', figsize=(400,225))
plt.title("Daily National AQI Mean",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('national_daily_average_aqi.pdf')

In [ ]:
grouped1 = df.groupby('Date')['AQI'].quantile(.25)

In [ ]:
grouped1.describe()

In [ ]:
grouped1.plot(kind='bar', title='national daily 25%', figsize=(400,225))
plt.title("Daily National AQI 25% Percentile",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('national_daily_25%_aqi.pdf')


In [ ]:
grouped2 = df.groupby('Date')['AQI'].median()

In [ ]:
grouped2.describe()

In [ ]:
grouped2.plot(kind='bar', title='national daily median', figsize=(400,225))
plt.title("Daily National AQI median",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('national_daily_median_aqi.pdf')

In [ ]:
grouped3 = df.groupby('Date')['AQI'].quantile(.75)

In [ ]:
grouped3.describe()

In [ ]:
grouped3.plot(kind='bar', title='national daily average', figsize=(400,225))
plt.title("Daily National AQI 75% Percentile",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('national_daily_75%_aqi.pdf')

## The followings are examples without loops:)

### 北京

In [ ]:
df=pd.read_csv("北京.csv")

In [ ]:
df.describe()

In [ ]:
grouped = df.groupby('Date')['AQI'].mean()

In [ ]:
grouped.describe()

In [ ]:
grouped.plot(kind='bar', title='Beijing daily average', figsize=(400,225))
plt.title("Beijing AQI Mean",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('Beijing_daily_average_aqi.pdf')

In [ ]:
grouped2 = df.groupby('Date')['AQI'].median()

In [ ]:
grouped2.describe()

In [ ]:
grouped2.plot(kind='bar', title='beijing daily median', figsize=(400,225))
plt.title("Beijing Daily AQI median",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('beijing_daily_median_aqi.pdf')

In [ ]:
grouped3 = df.groupby('Date')['AQI'].quantile(.75)

In [ ]:
grouped3.describe()

In [ ]:
grouped3.plot(kind='bar', title='beijing daily 75% aqi', figsize=(400,225))
plt.title("Beijing Daily AQI 75% Percentile",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('beijing_daily_75%_aqi.pdf')

In [ ]:
grouped3 = df.groupby('Date')['AQI'].quantile(.25)

In [ ]:
grouped3.describe()

In [ ]:
grouped3.plot(kind='bar', title='beijing daily 25% aqi', figsize=(400,225))
plt.title("Beijing Daily AQI 25% Percentile",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('beijing_daily_25%_aqi.pdf')

### 上海

In [ ]:
df=pd.read_csv("上海.csv")

In [ ]:
df.describe()

In [ ]:
grouped = df.groupby('Date')['AQI'].mean()

In [ ]:
grouped.describe()

In [ ]:
grouped.plot(kind='bar', title='Shanghai daily average', figsize=(400,225))
plt.title("Shanghai AQI Mean",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('Shanghai_daily_average_aqi.pdf')

In [ ]:
grouped2 = df.groupby('Date')['AQI'].median()

In [ ]:
grouped2.describe()

In [ ]:
grouped2.plot(kind='bar', title='shanghai daily median', figsize=(400,225))
plt.title("Shanghai Daily AQI median",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('Shanghai_daily_median_aqi.pdf')

In [ ]:
grouped3 = df.groupby('Date')['AQI'].quantile(.75)

In [ ]:
grouped3.plot(kind='bar', title='shanghai daily 75% aqi', figsize=(400,225))
plt.title("Shanghai Daily AQI 75% Percentile",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('Shanghai_daily_75%_aqi.pdf')

In [ ]:
grouped4 = df.groupby('Date')['AQI'].quantile(.25)

In [ ]:
grouped4.plot(kind='bar', title='shanghai daily 25% aqi', figsize=(400,225))
plt.title("Shanghai Daily AQI 25% Percentile",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('Shanghai_daily_25%_aqi.pdf')

### 云南

In [ ]:
df=pd.read_csv("云南.csv")

In [ ]:
grouped1 = df.groupby('Date')['AQI'].mean()

In [ ]:
grouped1.plot(kind='bar', title='Yunnan daily average', figsize=(400,225))
plt.title("Yunnan AQI Mean",fontsize=400,color='r')
plt.xlabel("Date", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.ylabel("AQI", fontdict={'family' : 'Times New Roman', 'size': 400})
plt.savefig('Yunnan_daily_average_aqi.pdf')